In [10]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()  # .env 파일의 환경변수 로드

api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

def generate_user_questions_prompt(task_name: str, task_summary: str, context: str) -> list:
    return [
        {
            "role": "system",
            "content": (
                "당신은 실제로 어떤 민원을 넣으려는 일반 시민입니다. 아래의 사무명, 사무개요, 그리고 민원안내 정보(context)를 참고하여, "
                "현실적인 배경이 있는 질문을 5개 작성하세요.\n\n"
                "각 질문은 반드시 다음 조건을 따라야 합니다:\n"
                "1. 실제 민원인의 처지에서 상황을 간단히 설명할 것\n"
                "2. 사무명에 포함된 핵심 키워드(예: 승강기, 농지, 건축물 등)를 반드시 문장 안에 포함할 것\n"
                "3. 질문은 아래 context에 포함된 항목의 정보만을 바탕으로 작성할 것 (context에 없는 정보는 언급하지 말 것)\n"
                "4. 질문만 읽어도 어떤 사무에 대한 내용인지 명확히 추정할 수 있을 것\n"
                "5. 막연한 표현(예: '서류', '신청서', '절차')만으로 구성된 질문은 지양할 것\n\n"
                "예시:\n"
                "- 사용 중이던 승강기에 대해 부과된 과징금을 분할 납부하려면 어떤 서류가 필요한가요?\n"
                "- 유기동물 인도 요청을 하려고 하는데, 농림축산검역본부 동물보호과에 어떻게 신청하나요?\n"
                "- 국민안전처 승강기정책과로 승강기 과징금 관련 신청서를 제출하려면 어떤 양식이 필요한가요?\n"
                "- 어업허가권 명의변경 신청을 하려면 수수료가 얼마나 드는지 궁금합니다.\n\n"
                "위 예시처럼, 질문은 반드시 구체적이며 context 안의 정보 기반이어야 합니다. "
                "이제 아래 정보를 참고하여 질문을 작성하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"[사무명]: {task_name}\n"
                f"[사무개요]: {task_summary}\n\n"
                f"[context 정보]:\n{context}"
            )
        }
    ]


def get_generated_questions(task_name, task_summary, context, idx=None, total=None):
    if idx is not None and total is not None:
        print(f"[{idx}/{total}] {task_name} 처리 중...")
    else:
        print(f"{task_name} 처리 중...")

    messages = generate_user_questions_prompt(task_name, task_summary, context)
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0.3
    )
    raw_output = response.choices[0].message.content
    return [q.strip("-• ").strip() for q in raw_output.strip().split("\n") if q.strip()]

In [11]:
import pandas as pd

qa_pairs = []

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

for i in range(1, 3000):
    print(f"[{i}/2999] 처리 중...")
    row = original_df.iloc[i]

    #사무명과 사무개요 유효성 확인,
    if isinstance(row.get("사무명"), str) and isinstance(row.get("사무개요"), str):
        task_name = row["사무명"]
        task_summary = row["사무개요"]

        # context 정보 구성
        context = (
            f"- 사무명: {row['사무명']}\n"
            f"- 사무개요: {row['사무개요']}\n"
            f"- 신청유형: {row.get('신청유형', '')}\n"
            f"- 신청자 자격: {row.get('신청자 자격', '')}\n"
            f"- 수수료: {row.get('수수료', '')}\n"
            f"- 수수료내역: {row.get('수수료내역', '')}\n"
            f"- 소관부처: {row.get('소관부처', '')}\n"
            f"- 담당부서: {row.get('담당부서', '')}\n"
            f"- 접수방법: {row.get('접수방법', '')}\n"
            f"- 접수/처리: {row.get('접수/처리', '')}\n"
            f"- 근거법령: {row.get('근거법령', '')}\n"
            f"- 신청서: {row.get('신청서', '')}\n"
            f"- 구비서류: {row.get('구비서류', '')}"
        )

        # GPT 호출로 질문 생성 (예: OpenAI API 등 사용)
        questions = get_generated_questions(task_name, task_summary, context)

        # 질문 저장
        for q in questions:
            print(f"\n질문: {q}")
            qa_pairs.append({
                "사무명": task_name,
                "질문": q,
            })

#DataFrame으로 저장,
new_df = pd.DataFrame(qa_pairs)

[1/2999] 처리 중...
공탁신청(공탁물지급청구 포함) 처리 중...

질문: 1. 채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와 신분증 외에 어떤 서류가 더 필요한가요?

질문: 2. 공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 알고 싶습니다.

질문: 3. 공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류가 다르다고 하던데, 구체적으로 어떤 서류가 필요한지 문의하고 싶습니다.

질문: 4. 공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는지 알려주실 수 있나요?

질문: 5. 공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인터넷으로도 신청이 가능한지 궁금합니다.
[2/2999] 처리 중...
친양자입양취소신고 처리 중...

질문: 1. 친양자 입양을 취소하려고 하는데, 가정법원의 허가를 받기 위해 필요한 서류는 어떤 것들이 있나요?

질문: 2. 친양자입양취소신고를 위해 가족관계등록부의 기본증명서와 친양자입양관계증명서를 제출해야 하는데, 이 서류를 어디서 발급받을 수 있나요?

질문: 3. 친양자 입양취소재판의 등본과 확정증명서를 준비해야 하는데, 이 서류를 발급받기 위한 절차는 어떻게 되나요?

질문: 4. 친양자입양취소신고를 우편으로 제출할 경우, 신고인의 신분증명서 사본은 어떻게 첨부해야 하나요?

질문: 5. 친양자입양취소신고를 위해 대법원의 가족관계등록과에 방문할 때, 어떤 신분증명서를 지참해야 하나요?
[3/2999] 처리 중...
친양자파양신고 처리 중...

질문: 1. 가정법원에서 친양자입양 파양 허가를 받은 후, 가족관계등록과에 신고하려고 하는데, 필요한 구비서류는 무엇인지 알고 싶습니다.

질문: 2. 친양자 파양재판의 확정증명서를 제출해야 하는데, 이 문서를 어떻게 발급받을 수 있는지 궁금합니다.

질문: 3. 친양자파양신고를 우편으로 제출할 경우, 신고인의 신분증명서 사본 외에 

KeyboardInterrupt: 

In [17]:
new_df = pd.DataFrame()
new_df = pd.DataFrame(qa_pairs)
new_df.to_csv('질답데이터(1-968).csv', index=False, encoding='utf-8-sig')
new_df

,사무명,질문
0,공탁신청(공탁물지급청구 포함),"1. 채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와..."
1,공탁신청(공탁물지급청구 포함),"2. 공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 ..."
2,공탁신청(공탁물지급청구 포함),"3. 공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류..."
3,공탁신청(공탁물지급청구 포함),"4. 공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는..."
4,공탁신청(공탁물지급청구 포함),"5. 공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인..."
...,...,...
4835,항로표지위탁관리업 등록,1. 항로표지위탁관리업 등록을 위해 필요한 구비서류 중 보유한 시설 및 장비명세서는...
4836,항로표지위탁관리업 등록,"2. 항로표지위탁관리업을 영위하고자 하는데, 자본금을 증명하는 서류는 어떤 내용을 ..."
4837,항로표지위탁관리업 등록,"3. 항로표지위탁관리업 등록 신청 시, 사설항로표지관리원의 자격을 증명하는 서류는 ..."
4838,항로표지위탁관리업 등록,"4. 항로표지위탁관리업 등록을 위해 정관을 제출해야 하는 경우, 법인등기부등본과 함..."


In [18]:
import pandas as pd

qa_pairs = []

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

for i in range(969, 3000):
    print(f"[{i}/2999] 처리 중...")
    row = original_df.iloc[i]

    #사무명과 사무개요 유효성 확인,
    if isinstance(row.get("사무명"), str) and isinstance(row.get("사무개요"), str):
        task_name = row["사무명"]
        task_summary = row["사무개요"]

        # context 정보 구성
        context = (
            f"- 사무명: {row['사무명']}\n"
            f"- 사무개요: {row['사무개요']}\n"
            f"- 신청유형: {row.get('신청유형', '')}\n"
            f"- 신청자 자격: {row.get('신청자 자격', '')}\n"
            f"- 수수료: {row.get('수수료', '')}\n"
            f"- 수수료내역: {row.get('수수료내역', '')}\n"
            f"- 소관부처: {row.get('소관부처', '')}\n"
            f"- 담당부서: {row.get('담당부서', '')}\n"
            f"- 접수방법: {row.get('접수방법', '')}\n"
            f"- 접수/처리: {row.get('접수/처리', '')}\n"
            f"- 근거법령: {row.get('근거법령', '')}\n"
            f"- 신청서: {row.get('신청서', '')}\n"
            f"- 구비서류: {row.get('구비서류', '')}"
        )

        # GPT 호출로 질문 생성 (예: OpenAI API 등 사용)
        questions = get_generated_questions(task_name, task_summary, context)

        # 질문 저장
        for q in questions:
            print(f"\n질문: {q}")
            qa_pairs.append({
                "사무명": task_name,
                "질문": q,
            })

[969/2999] 처리 중...
공유수면원상회복 의무면제(점용, 사용) 처리 중...

질문: 1. 공유수면 원상회복 의무 면제를 신청하고 싶은데, 신청구역을 표시한 2만5천분의 1의 지형도를 어떻게 준비해야 하나요?

질문: 2. 제가 점용하고 있는 공유수면에 대해 원상회복이 불가능한 사유가 있는데, 이를 증명하기 위한 서류는 어떤 것이 필요한가요?

질문: 3. 공유수면 원상회복 의무 면제를 신청하기 위해 제출해야 하는 기본서류 중 법인등기사항증명서는 어디서 발급받을 수 있나요?

질문: 4. 해양수산부 해양공간정책과에 공유수면 원상회복 의무 면제 신청서를 제출하려고 하는데, 인터넷으로 신청하는 방법은 무엇인가요?

질문: 5. 공유수면 원상회복 의무 면제를 신청한 후 처리 기간이 14일이라고 하는데, 이 기간 동안 어떤 절차가 진행되는지 알고 싶습니다.
[970/2999] 처리 중...
어장정화·정비업 등록신청 처리 중...

질문: 1. 어장정화·정비업을 시작하려고 하는데, 필요한 구비서류 중에서 기술인력현황서류사본은 어떻게 준비해야 하나요?

질문: 2. 어장정화·정비업 등록신청을 위해 임차한 선박의 임차권을 증명하는 서류는 어떤 형식으로 제출해야 하는지 궁금합니다.

질문: 3. 어장정화·정비업 등록을 위해 필요한 기본서류 중 자본금 보유현황증명서류는 어떤 내용을 포함해야 하나요?

질문: 4. 어장정화·정비업 등록신청을 하려면 정관을 제출해야 하는데, 정관 작성 시 주의해야 할 점이 있을까요?

질문: 5. 어장정화·정비업 등록신청을 위해 법인등기사항증명서를 제출해야 하는 경우, 이 서류는 어디서 발급받을 수 있나요?
[971/2999] 처리 중...
선박재검사(재검정, 재확인) 처리 중...

질문: 1. 최근에 선박 검사를 받았는데, 검사 결과에 불복하여 선박 재검사를 신청하고 싶습니다. 이 경우, 어떤 절차를 따라야 하나요?

질문: 2. 선박 재검정 신청을 위해 필요한 서류가 없다고 들었는데, 그럼에도 불구하고 재검사를 신청할 때 주의해야

In [20]:
qa_pairs

new_df2 = pd.DataFrame()
new_df2 = pd.DataFrame(qa_pairs)
new_df2.to_csv('질답데이터(969-3000).csv', index=False, encoding='utf-8-sig')
new_df2

,사무명,질문
0,"공유수면원상회복 의무면제(점용, 사용)","1. 공유수면 원상회복 의무 면제를 신청하고 싶은데, 신청구역을 표시한 2만5천분의..."
1,"공유수면원상회복 의무면제(점용, 사용)","2. 제가 점용하고 있는 공유수면에 대해 원상회복이 불가능한 사유가 있는데, 이를 ..."
2,"공유수면원상회복 의무면제(점용, 사용)",3. 공유수면 원상회복 의무 면제를 신청하기 위해 제출해야 하는 기본서류 중 법인등...
3,"공유수면원상회복 의무면제(점용, 사용)",4. 해양수산부 해양공간정책과에 공유수면 원상회복 의무 면제 신청서를 제출하려고 하...
4,"공유수면원상회복 의무면제(점용, 사용)","5. 공유수면 원상회복 의무 면제를 신청한 후 처리 기간이 14일이라고 하는데, 이..."
...,...,...
10150,영양사면허증 재교부,"1. 영양사면허증을 분실했는데, 재교부를 받기 위해 어떤 증명서류를 제출해야 하나요?"
10151,영양사면허증 재교부,"2. 영양사면허증에 기재된 제 이름이 변경되어 재교부를 신청하려고 하는데, 가족관계..."
10152,영양사면허증 재교부,"3. 영양사면허증 재교부 신청을 위해 필요한 사진 규격이 어떻게 되며, 몇 장을 제..."
10153,영양사면허증 재교부,4. 주민등록번호가 변경된 경우 영양사면허증 재교부를 위해 어떤 추가 서류를 제출해...


In [21]:
# 두 데이터프레임을 행 방향으로 합치기
merged_df = pd.concat([new_df, new_df2], ignore_index=True)
merged_df.to_csv('질답데이터(1-3000).csv', index=False, encoding='utf-8-sig')
merged_df

,사무명,질문
0,공탁신청(공탁물지급청구 포함),"1. 채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와..."
1,공탁신청(공탁물지급청구 포함),"2. 공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 ..."
2,공탁신청(공탁물지급청구 포함),"3. 공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류..."
3,공탁신청(공탁물지급청구 포함),"4. 공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는..."
4,공탁신청(공탁물지급청구 포함),"5. 공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인..."
...,...,...
14990,영양사면허증 재교부,"1. 영양사면허증을 분실했는데, 재교부를 받기 위해 어떤 증명서류를 제출해야 하나요?"
14991,영양사면허증 재교부,"2. 영양사면허증에 기재된 제 이름이 변경되어 재교부를 신청하려고 하는데, 가족관계..."
14992,영양사면허증 재교부,"3. 영양사면허증 재교부 신청을 위해 필요한 사진 규격이 어떻게 되며, 몇 장을 제..."
14993,영양사면허증 재교부,4. 주민등록번호가 변경된 경우 영양사면허증 재교부를 위해 어떤 추가 서류를 제출해...


In [22]:
# 기존 merged_df가 이미 존재한다고 가정
df_new = pd.read_excel('3000_6021_민원질문데이터셋.xlsx')
merged_df = pd.concat([merged_df, df_new], ignore_index=True)

In [24]:
pd.set_option('display.max_colwidth', None)  # 또는 0.25.0 이상에서는 'None', 그 이하 버전은 -1
merged_df

,사무명,질문
0,공탁신청(공탁물지급청구 포함),"1. 채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와 신분증 외에 어떤 서류가 더 필요한가요?"
1,공탁신청(공탁물지급청구 포함),"2. 공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 알고 싶습니다."
2,공탁신청(공탁물지급청구 포함),"3. 공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류가 다르다고 하던데, 구체적으로 어떤 서류가 필요한지 문의하고 싶습니다."
3,공탁신청(공탁물지급청구 포함),"4. 공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는지 알려주실 수 있나요?"
4,공탁신청(공탁물지급청구 포함),"5. 공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인터넷으로도 신청이 가능한지 궁금합니다."
...,...,...
30100,고충민원 (공통),"1. 고충민원으로 특정한 행위를 행정기관에 제기하고 싶은데, 진정서를 작성할 때 어떤 내용을 포함해야 하나요?"
30101,고충민원 (공통),"2. 고충민원 신청을 위해 진정서를 제출하려고 하는데, 기본서류 외에 추가로 필요한 서류가 있나요?"
30102,고충민원 (공통),"3. 고충민원으로 민원을 제기할 때, 전화나 인터넷을 통해 신청하는 방법은 어떻게 되나요?"
30103,고충민원 (공통),"4. 고충민원에 대한 처리 기간이 7일이라고 하는데, 이 기간 내에 어떤 절차가 진행되는지 알고 싶습니다."


In [25]:
merged_df.to_csv('merged_df.csv', index=False, encoding='utf-8-sig')

In [26]:
#앞에 번호 삭제

pattern = r'^\d+\. '
not_matched = merged_df[~merged_df['질문'].str.match(pattern)]

if not not_matched.empty:
    print('번호 패턴이 아닌 질문 예시:')
    print(not_matched['질문'].unique())

In [27]:
import re

def remove_number_prefix(text):
    if re.match(r'^\d+\. ', str(text)):
        return text[3:]
    return text

merged_df['질문'] = merged_df['질문'].apply(remove_number_prefix)

In [28]:
merged_df

,사무명,질문,질문_전처리
0,공탁신청(공탁물지급청구 포함),"1. 채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와 신분증 외에 어떤 서류가 더 필요한가요?","채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와 신분증 외에 어떤 서류가 더 필요한가요?"
1,공탁신청(공탁물지급청구 포함),"2. 공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 알고 싶습니다.","공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 알고 싶습니다."
2,공탁신청(공탁물지급청구 포함),"3. 공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류가 다르다고 하던데, 구체적으로 어떤 서류가 필요한지 문의하고 싶습니다.","공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류가 다르다고 하던데, 구체적으로 어떤 서류가 필요한지 문의하고 싶습니다."
3,공탁신청(공탁물지급청구 포함),"4. 공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는지 알려주실 수 있나요?","공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는지 알려주실 수 있나요?"
4,공탁신청(공탁물지급청구 포함),"5. 공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인터넷으로도 신청이 가능한지 궁금합니다.","공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인터넷으로도 신청이 가능한지 궁금합니다."
...,...,...,...
30100,고충민원 (공통),"1. 고충민원으로 특정한 행위를 행정기관에 제기하고 싶은데, 진정서를 작성할 때 어떤 내용을 포함해야 하나요?","고충민원으로 특정한 행위를 행정기관에 제기하고 싶은데, 진정서를 작성할 때 어떤 내용을 포함해야 하나요?"
30101,고충민원 (공통),"2. 고충민원 신청을 위해 진정서를 제출하려고 하는데, 기본서류 외에 추가로 필요한 서류가 있나요?","고충민원 신청을 위해 진정서를 제출하려고 하는데, 기본서류 외에 추가로 필요한 서류가 있나요?"
30102,고충민원 (공통),"3. 고충민원으로 민원을 제기할 때, 전화나 인터넷을 통해 신청하는 방법은 어떻게 되나요?","고충민원으로 민원을 제기할 때, 전화나 인터넷을 통해 신청하는 방법은 어떻게 되나요?"
30103,고충민원 (공통),"4. 고충민원에 대한 처리 기간이 7일이라고 하는데, 이 기간 내에 어떤 절차가 진행되는지 알고 싶습니다.","고충민원에 대한 처리 기간이 7일이라고 하는데, 이 기간 내에 어떤 절차가 진행되는지 알고 싶습니다."


In [29]:
merged_df = merged_df.drop(columns=['질문'])

In [30]:
merged_df

,사무명,질문_전처리
0,공탁신청(공탁물지급청구 포함),"채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와 신분증 외에 어떤 서류가 더 필요한가요?"
1,공탁신청(공탁물지급청구 포함),"공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 알고 싶습니다."
2,공탁신청(공탁물지급청구 포함),"공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류가 다르다고 하던데, 구체적으로 어떤 서류가 필요한지 문의하고 싶습니다."
3,공탁신청(공탁물지급청구 포함),"공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는지 알려주실 수 있나요?"
4,공탁신청(공탁물지급청구 포함),"공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인터넷으로도 신청이 가능한지 궁금합니다."
...,...,...
30100,고충민원 (공통),"고충민원으로 특정한 행위를 행정기관에 제기하고 싶은데, 진정서를 작성할 때 어떤 내용을 포함해야 하나요?"
30101,고충민원 (공통),"고충민원 신청을 위해 진정서를 제출하려고 하는데, 기본서류 외에 추가로 필요한 서류가 있나요?"
30102,고충민원 (공통),"고충민원으로 민원을 제기할 때, 전화나 인터넷을 통해 신청하는 방법은 어떻게 되나요?"
30103,고충민원 (공통),"고충민원에 대한 처리 기간이 7일이라고 하는데, 이 기간 내에 어떤 절차가 진행되는지 알고 싶습니다."


In [31]:
original_df

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN
1,9740000-0009,공탁신청(공탁물지급청구 포함),"공탁 신청, 공탁금의 지급을 위한 민원입니다. 공탁은 공탁자가 법령에 규정된 원인에 따라 금전, 유가증권, 그 밖의 물품을 공탁소에 맡기고, 일정한 자가 수령하도록 함으로써 채무의 변제, 손해의 담보, 집행목적물의 관리 및 집행당사자에의 교부, 목적물의 보관 등의 목적을 달성하게 하는 제도입니다. ※ 공탁의 종류에 따라 공탁신청시 또는 지급청구시 신청서와 첨부서류가 다르므로, 구체적인 첨부서류 등은 해당 공탁소로 문의하여 주시기 바랍니다.",승인,누구나신청가능,수수료 없음,수수료 없음,대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방법원.지원(0일)\n*경유:\n*협의:\n*처리:지방법원.지원(0일),"1. 공탁법(제4조, 제9조) \n2. 공탁규칙(제20조, 제21조, 제23조, 제24조, 제32조, 제33조, 제34조, 제37조, 제38조) \n3. 공탁사무 문서양식에 관한 예규(제1조, 제2조)",금전공탁서(변제 등) 공탁사무 문서양식에 관한 예규 : 별지서식 1호의 1\n,NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 인감증명서 신분증 가압류,압류결정문 사본 담보제공명령서 사본\n< 공무원확인사항 >\n(2) 외국인등록사실증명\n(3) 국내거소신고사실증명\n(4) 사업자등록증명\n(5) 주민등록등(초)본"
2,9740000-0008,친양자입양취소신고,가정법원의 허가를 얻어 친양자 입양을 취소하기 위해 신청하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 즉시)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(즉시),"1. 가족관계의 등록 등에 관한 법률(제69조, 제70조)","제9호, 친양자 입양취소신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자입양취소당사자의 가족관계등록부의 기본증명서, 친양자입양관계증명서 각1통(전산정보처리에 의해 확인할 수 없는 경우에 한함)\n(2) 친양자 입양취소재판의 등본 및 확정증명 각1부\n(3) 신고인의 신분증명서 사본 첨부(우편신고의 경우에 한함)\n(4) 신고인이 출석한 경우의 신고인 신분증명서, 제출인이 출석한 경우 제출인의 신분증명서\n< 공무원확인사항 >\n(5) 가족관계등록부"
3,9740000-0007,친양자파양신고,가정법원의 허가를 얻어 친양자입양을 파양하고자 하는 사람이 재판의 확정일부터 1개월 이내에 가족관계등록 관서의 장에게 신고하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(지체없이),1. 가족관계의 등록 등에 관한 법률(제69조),"제7호, 친양자파양신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자파양당사자의 가족관계등록부의 기본증명서, 친양자입양관계증명서 각1통(전산정보처리조지에 의하여 그 내용을 확인할 수 없는 경우에 한함)\n(2) 친양자 파양재판의 등본 및 확정증명 각 1부\n(3) 신고인의 신분증명서 사본 첨부(우편신고의 경우에 한함)\n(4) 신고인이 출석한 경우에는 신고인의 신분증명서, 제출인이 출석한 경우에는 제출인의 신분증명서\n< 공무원확인사항 >\n(5) 가족관계등록부"
4,9740000-0006,성·본계속사용신고,부가 혼인외 자를 인지할 때 인지 전의 자녀의 성과 본을 유지할 경우 그 취지를 가족관계등록 관서의 장에게 신고하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음(총 0일)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(0일),1. 가족관계의 등록 등에 관한 법률(제55조 제1항),"제3호, 성·본계속사용신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 성·본계속사용자의 가족관계등록부의 가족관계증명서 1통. (전산정보처리조직에 의하여 그 내용을 확인할 수 없을 경우에 한함)\n(2) 성.본계속사용 법원 재판서 등본 1부\n(3) 성.본계속사용 협의사실을 증명하는 서류 1부 및 신고인의 신분증 지참(부모가 협의한 경 우로서 부모 중 일방이 신고할 경우에 한함)\n(4) 신고인의 신분증명서 사본 1부(우편으로 신고하는 경우에 한함) ※신고인이 출석한 경우에는 신고인의 신분증명서, 제출인이 출석한 경우에는 제출인의 신분증명서\n(5) 부모의 협의에 의한 경우의 추가 구비서류 ( 붙임의 신고서 서식 참조)\n< 공무원확인사항 >\n(6) 가족관계등록부"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6017,0000000-0008,재직(퇴직·경력)증명,"공공기관에서 근무하였던 자 또는 근무하고 있는 자가 경력사항을 증명 받을 때 혹은 퇴직할 때 신청하는 민원사무입니다.(현재 온라인 신청은 중앙부처(퇴직자) 및 지방자치단체 경력증명만 신청 가능합니다.) e-사람 독립망 운영기관(국방부, 방위사업청, 국세청, 경찰청, 대통령비서실, 대통령경호처)은 퇴직자 경력증명서를 온라인 신청할 수 없습니다. 해당 기관의 담당자에게 별도 요청바랍니다.",증명,본인또는대리인신청,조례별 수수료,조례별 수수료,공통,공통,"방문, 우편, 전화, FAX, 인터넷, 현장제출 및 기타",1. 경력증명서신청(총 0일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:전행정기관(0일)\n2. 경력증명신청_중앙부처(총 0일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:전행정기관(0일),1. 공무원 인사기록ㆍ통계 및 인사사무 처리 규정(제32조) \n2. 공무원 인사·성과기록 및 전자인사관리 규칙() \n3. 무기계약 및 1년이상 기간제근로자 관리규정(),"재직증명서,경력증명서 공무원 인사·성과기록 및 전자인사관리 규칙 : 별지서식 42,43호\n",NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 재직증명서\n(2) 경력증명서
6018,0000000-0004,확인 (공통),각급 행정기관에서 보유하고 있는 자료 또는 공부상의 기록을 확인하기 위하여 열람 또는 발급해주는 민원사무입니다.,확인,본인또는대리인신청,수수료 없음,수수료 없음,공통,공통,"방문, 우편, 전화, FAX, 인터넷, 현장제출 및 기타",1. 유형없음(총 0일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:전행정기관(0일),1. 개별법령(),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 1.확인서
6019,0000000-0003,질의 (공통),"법령·제도·절차 등 행정업무에 관한 질의 또는 상담형식을 통한 설명이나 해석을 요구하고자 할때 신청하는 민원사무입니다. ※ 본인 확인이 불필요한 민원사항, 민원인의 권리형성·의무발생에 직접 변동을 가져다주지 않는 단순질의의 경우 전화·우편·팩스·인터넷으로 신청할 수 있습니다",질의,누구나신청가능,수수료 없음,수수료 없음,공통,공통,"방문, 우편, 전화, FAX, 인터넷",1. 단순질의(총 7일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:전행정기관(7일)\n2. 법령질의(총 14일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:전행정기관(14일),1. 민원 처리에 관한 법률(),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 1.질의서
6020,0000000-0002,건의 (공통),"정부시책이나 행정제도 및 운영의 개선에 관한 사항을 건의하기 위한 민원사무입니다. ※ 본인 확인이 불필요한 민원사항, 민원인의 권리형성·의무발생에 직접변

In [32]:
import pandas as pd

qa_pairs = []

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

for i in range(0, 1):
    print(f"[{i}/2999] 처리 중...")
    row = original_df.iloc[i]

    #사무명과 사무개요 유효성 확인,
    if isinstance(row.get("사무명"), str) and isinstance(row.get("사무개요"), str):
        task_name = row["사무명"]
        task_summary = row["사무개요"]

        # context 정보 구성
        context = (
            f"- 사무명: {row['사무명']}\n"
            f"- 사무개요: {row['사무개요']}\n"
            f"- 신청유형: {row.get('신청유형', '')}\n"
            f"- 신청자 자격: {row.get('신청자 자격', '')}\n"
            f"- 수수료: {row.get('수수료', '')}\n"
            f"- 수수료내역: {row.get('수수료내역', '')}\n"
            f"- 소관부처: {row.get('소관부처', '')}\n"
            f"- 담당부서: {row.get('담당부서', '')}\n"
            f"- 접수방법: {row.get('접수방법', '')}\n"
            f"- 접수/처리: {row.get('접수/처리', '')}\n"
            f"- 근거법령: {row.get('근거법령', '')}\n"
            f"- 신청서: {row.get('신청서', '')}\n"
            f"- 구비서류: {row.get('구비서류', '')}"
        )

        # GPT 호출로 질문 생성 (예: OpenAI API 등 사용)
        questions = get_generated_questions(task_name, task_summary, context)

        # 질문 저장
        for q in questions:
            print(f"\n질문: {q}")
            qa_pairs.append({
                "사무명": task_name,
                "질문": q,
            })

#DataFrame으로 저장,
new_df = pd.DataFrame(qa_pairs)

[0/2999] 처리 중...
토지임야대장등본교부 다량신청 처리 중...

질문: 1. 여러 번지의 토지임야대장등본을 한 번에 신청하고 싶은데, 국토교통부 공간정보제도과에 어떤 절차를 따라야 하나요?

질문: 2. 토지임야대장등본을 다량으로 신청할 경우, 인터넷을 통해 접수하는 방법에 대해 자세히 알고 싶습니다.

질문: 3. 여러 번지의 토지임야대장등본을 교부받기 위해 필요한 구비서류가 있는지 궁금합니다.

질문: 4. 토지임야대장등본을 다량으로 신청할 때, 수수료가 발생하지 않는다고 들었는데, 이 점에 대해 확인할 수 있을까요?

질문: 5. 국토교통부 공간정보제도과에 토지임야대장등본을 다량으로 신청할 때, 특별한 자격 요건이 필요한지 알고 싶습니다.


In [33]:
new_df3 = pd.DataFrame()
new_df3 = pd.DataFrame(qa_pairs)

In [34]:
new_df3

,사무명,질문
0,토지임야대장등본교부 다량신청,"1. 여러 번지의 토지임야대장등본을 한 번에 신청하고 싶은데, 국토교통부 공간정보제도과에 어떤 절차를 따라야 하나요?"
1,토지임야대장등본교부 다량신청,"2. 토지임야대장등본을 다량으로 신청할 경우, 인터넷을 통해 접수하는 방법에 대해 자세히 알고 싶습니다."
2,토지임야대장등본교부 다량신청,3. 여러 번지의 토지임야대장등본을 교부받기 위해 필요한 구비서류가 있는지 궁금합니다.
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청할 때, 수수료가 발생하지 않는다고 들었는데, 이 점에 대해 확인할 수 있을까요?"
4,토지임야대장등본교부 다량신청,"5. 국토교통부 공간정보제도과에 토지임야대장등본을 다량으로 신청할 때, 특별한 자격 요건이 필요한지 알고 싶습니다."


In [38]:
merged_df.rename(columns={'질문_전처리': '질문'}, inplace=True)

merged_df

,사무명,질문
0,공탁신청(공탁물지급청구 포함),"채무 변제를 위한 금전공탁을 신청하려고 하는데, 필요한 구비서류로 인감증명서와 신분증 외에 어떤 서류가 더 필요한가요?"
1,공탁신청(공탁물지급청구 포함),"공탁금 지급청구를 하려면 어떤 절차를 따라야 하며, 필요한 신청서는 무엇인지 알고 싶습니다."
2,공탁신청(공탁물지급청구 포함),"공탁소에 방문하여 공탁신청을 하려는데, 공탁의 종류에 따라 제출해야 하는 서류가 다르다고 하던데, 구체적으로 어떤 서류가 필요한지 문의하고 싶습니다."
3,공탁신청(공탁물지급청구 포함),"공탁신청을 위해 금전공탁서를 작성해야 하는데, 이 서식은 어디서 받을 수 있는지 알려주실 수 있나요?"
4,공탁신청(공탁물지급청구 포함),"공탁금 지급청구를 하려면 대법원 사법등기국에 직접 방문해야 하는지, 아니면 인터넷으로도 신청이 가능한지 궁금합니다."
...,...,...
30100,고충민원 (공통),"고충민원으로 특정한 행위를 행정기관에 제기하고 싶은데, 진정서를 작성할 때 어떤 내용을 포함해야 하나요?"
30101,고충민원 (공통),"고충민원 신청을 위해 진정서를 제출하려고 하는데, 기본서류 외에 추가로 필요한 서류가 있나요?"
30102,고충민원 (공통),"고충민원으로 민원을 제기할 때, 전화나 인터넷을 통해 신청하는 방법은 어떻게 되나요?"
30103,고충민원 (공통),"고충민원에 대한 처리 기간이 7일이라고 하는데, 이 기간 내에 어떤 절차가 진행되는지 알고 싶습니다."


In [40]:
import re

def remove_number_prefix(text):
    if re.match(r'^\d+\. ', str(text)):
        return text[3:]
    return text

new_df3['질문'] = new_df3['질문'].apply(remove_number_prefix)

new_df3

,사무명,질문
0,토지임야대장등본교부 다량신청,"여러 번지의 토지임야대장등본을 한 번에 신청하고 싶은데, 국토교통부 공간정보제도과에 어떤 절차를 따라야 하나요?"
1,토지임야대장등본교부 다량신청,"토지임야대장등본을 다량으로 신청할 경우, 인터넷을 통해 접수하는 방법에 대해 자세히 알고 싶습니다."
2,토지임야대장등본교부 다량신청,여러 번지의 토지임야대장등본을 교부받기 위해 필요한 구비서류가 있는지 궁금합니다.
3,토지임야대장등본교부 다량신청,"토지임야대장등본을 다량으로 신청할 때, 수수료가 발생하지 않는다고 들었는데, 이 점에 대해 확인할 수 있을까요?"
4,토지임야대장등본교부 다량신청,"국토교통부 공간정보제도과에 토지임야대장등본을 다량으로 신청할 때, 특별한 자격 요건이 필요한지 알고 싶습니다."


In [41]:
# new_df3을 맨 앞에, merged_df를 그 뒤에 붙임
result_df = pd.concat([new_df3, merged_df], ignore_index=True)

result_df

,사무명,질문
0,토지임야대장등본교부 다량신청,"여러 번지의 토지임야대장등본을 한 번에 신청하고 싶은데, 국토교통부 공간정보제도과에 어떤 절차를 따라야 하나요?"
1,토지임야대장등본교부 다량신청,"토지임야대장등본을 다량으로 신청할 경우, 인터넷을 통해 접수하는 방법에 대해 자세히 알고 싶습니다."
2,토지임야대장등본교부 다량신청,여러 번지의 토지임야대장등본을 교부받기 위해 필요한 구비서류가 있는지 궁금합니다.
3,토지임야대장등본교부 다량신청,"토지임야대장등본을 다량으로 신청할 때, 수수료가 발생하지 않는다고 들었는데, 이 점에 대해 확인할 수 있을까요?"
4,토지임야대장등본교부 다량신청,"국토교통부 공간정보제도과에 토지임야대장등본을 다량으로 신청할 때, 특별한 자격 요건이 필요한지 알고 싶습니다."
...,...,...
30105,고충민원 (공통),"고충민원으로 특정한 행위를 행정기관에 제기하고 싶은데, 진정서를 작성할 때 어떤 내용을 포함해야 하나요?"
30106,고충민원 (공통),"고충민원 신청을 위해 진정서를 제출하려고 하는데, 기본서류 외에 추가로 필요한 서류가 있나요?"
30107,고충민원 (공통),"고충민원으로 민원을 제기할 때, 전화나 인터넷을 통해 신청하는 방법은 어떻게 되나요?"
30108,고충민원 (공통),"고충민원에 대한 처리 기간이 7일이라고 하는데, 이 기간 내에 어떤 절차가 진행되는지 알고 싶습니다."
